In [ ]:
library(SPARK)
library(anndata)

In [ ]:
for(i in c(0:9)){
    adata <- read_h5ad(glue::glue("../../data/simulation/svgs/adata_rep_{i}.h5ad"))
    counts = t(as.matrix(adata$layers[['counts']]))
    info <- as.data.frame(adata$obsm[['spatial']])
    rownames(info) <- colnames(counts)
    colnames(info) <- c("x", "y")

    sparkX <- sparkx(counts, info[, 1:2], numCores=10,option="mixture")
    
    df_var <- adata$var
    df_var$spatially_variable <- as.integer(df_var$spatially_variable)

    df <- as.data.frame(sparkX$res_mtest)
    df <- df[rownames(df_var), ]
    df$spatially_variable <- as.integer(df_var$spatially_variable)
    df <- df[, c("adjustedPval", "spatially_variable")]
    
    write.csv(df, file=glue::glue("./svgs/rep_{i}.csv"), quote=FALSE)

}